In [ ]:
from scripts.utils_2 import *
from tensorflow import keras
import tensorflow as tf
from time import strftime

In [ ]:
## Preprocess
for file in os.listdir("data/renamed"):
    preprocess(file)

In [ ]:
## Windowing
for file in os.listdir("data/preprocessed"):
    window(file)

In [ ]:
## Balancing
for file in tqdm(os.listdir("data/windowed")):
  balance(file)

In [ ]:
accs = []
aucs = []
fs = []
precs = []
recs = []
dir = "data/windowed/"
for file in os.listdir(dir):
    print(file)
    df = pd.read_csv(dir+file)
    p,s,w = class_count(df)

    test_labels = np.array(df.pop('Class'))

    test_features = np.array(df)
    model = keras.models.load_model('./model')
    BATCH_SIZE=200
    hln = 200
    test_predictions_baseline = model.predict(test_features, batch_size=BATCH_SIZE)
    test_predictions_baseline.argmax(axis=1)
    baseline_results = model.evaluate(test_features, tf.one_hot(test_labels,depth=3),
                                        batch_size=BATCH_SIZE, verbose=0,return_dict=True)
    # plot_cm(tf.one_hot(test_labels,depth=3).numpy().argmax(axis=1),test_predictions_baseline.argmax(axis=1),baseline_results,hln,file)
    f = 2*(baseline_results['precision']*baseline_results['recall'])/(baseline_results['precision']+baseline_results['recall'])
    accs.append(baseline_results['categorical_accuracy'])
    aucs.append(baseline_results['auc'])
    fs.append(f)
    precs.append(baseline_results['precision'])
    recs.append(baseline_results['recall'])

    date = strftime('%X %x').replace("/","").split()
    plt.savefig("figures/"+str(date[1])+"@"+str(date[0][:5].replace(":",""))+"_"+str(hln)+"_"+file.replace(".csv","")+".jpg",bbox_inches='tight')

In [ ]:
sns.displot(aucs,bins=20,kde=True)
sns.displot(accs,bins=20,kde=True)
sns.displot(fs,bins=20,kde=True)
sns.displot(precs,bins=20,kde=True)
sns.jointplot(precs,recs,kind="kde",rug=True)

g = sns.JointGrid(precs,recs)
g.plot_joint(sns.histplot)
g.plot_marginals(sns.boxplot)


In [ ]:
i=0
plt.figure(figsize=(50,50)) # specifying the overall grid size
import matplotlib.image as mpimg
for file in os.listdir('figures'):
    img = mpimg.imread("figures/"+file)
    plt.subplot(5,5,i+1)    # the number of images in the grid is 5*5 (25)
    i=i+1
    plt.imshow(img)
plt.savefig('out.jpg',dpi=100,bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import time

fig = plt.figure()
ax1 = fig.add_subplot(1,1,1)

def animate(i):
    df = pd.read_csv("data/windowed/2_preprocessed_windowed.csv")

    dataArray = df.iloc
    xar = range(100)
    yar = df['1'][:1000]
    for eachLine in dataArray:
        if len(eachLine)>1:
            yar.append(df['1'][:100])
    ax1.clear()
    ax1.plot(xar,yar)
ani = animation.FuncAnimation(fig, animate, interval=1000)
plt.show()